In [1]:
!conda env list

# conda environments:
#
base                     /home/matthias/anaconda3
base20220121             /home/matthias/anaconda3/envs/base20220121
hf                       /home/matthias/anaconda3/envs/hf
hf20220408            *  /home/matthias/anaconda3/envs/hf20220408
nlu                      /home/matthias/anaconda3/envs/nlu
nlu20220407              /home/matthias/anaconda3/envs/nlu20220407
pytorch                  /home/matthias/anaconda3/envs/pytorch
pytorch20220121          /home/matthias/anaconda3/envs/pytorch20220121



1. ***make copy of this notebook and inspect the dataset dict***
1. ***simplify the dataset as much as possible while still running the entire notebook successfully***
1. ***cast Vorarlberg data into the format of that simplified dataset***
1. ***cross fingers and run notebook with Vorarlberg data***


# Token Classification
**See also [Colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb#scrollTo=G7JKGAypBaPR), [Hugging Face](https://huggingface.co/docs/transformers/main/en/tasks/token_classification), [StackOverflow](https://stackoverflow.com/questions/39649102/how-do-i-select-which-gpu-to-run-a-job-on), and [Data Science StackExchange](https://datascience.stackexchange.com/questions/20179/what-is-the-advantage-of-keeping-batch-size-a-power-of-2).**

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import transformers
print(transformers.__version__)

4.16.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

## Fine-tuning a model on a token classification task
In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token. Named-entity recognition (NER) is one of the most common token classification tasks.

We will see how to easily load a dataset for this tasks and use the `Trainer` API to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [3]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
#model_checkpoint = "philschmid/gbert-base-germaner"                  # requires tensorflow
model_checkpoint = "fhswf/bert_de_ner" # model and autotokenizer mismatch
#model_checkpoint = "KoichiYasuoka/bert-base-german-upos"
batch_size = 8 # changed from 16 (8 works)

### Loading the dataset
We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [4]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [5]:
datasets = load_dataset("conll2003")

Reusing dataset conll2003 (/home/matthias/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee)


  0%|          | 0/3 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3454
    })
})

In [54]:
datasets2 = datasets # backup for later (if needed)

In [62]:
datasets = {}
# rebuild "datasets" from "datasets2" but without "num_rows"
data_sets = ["train", "validation", "test"]
feats = ["id", "tokens", "pos_tags", "chunk_tags", "ner_tags"]
for data_set in data_sets:
    datasets[data_set] = {}
    for feat in feats:
        datasets[data_set][feat] = datasets2[data_set][feat]
datasets["train"]["id"]

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138'

In [35]:
dataset2 = datasets # backup for later
#print(datasets["train"])
dataset_choice = "test"
print(datasets[dataset_choice].features)
print()
datasets[dataset_choice]["tokens"][-1]

{'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'pos_tags': Sequence(feature=ClassLabel(num_classes=47, names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None), 'chunk_tags': Sequence(feature=ClassLabel(num_classes=23, names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}



[]

In [46]:
data_sets = ["train", "validation", "test"]
feats = ["id", "tokens", "pos_tags", "chunk_tags", "ner_tags"]
for data_set in data_sets:
    print(data_set)

train
validation
test


In [53]:
data_sets = ["train", "validation", "test"]
feats = ["id", "tokens", "pos_tags", "chunk_tags", "ner_tags"]
for data_set in data_sets:
    print("\n{}".format(data_set))
    for feat in feats:
        print(feat, end="\t")
        print(datasets[data_set][feat][-2])


train
id	14040
tokens	['Swansea', '1', 'Lincoln', '2']
pos_tags	[21, 11, 22, 11]
chunk_tags	[11, 12, 12, 12]
ner_tags	[3, 0, 3, 0]

validation
id	3249
tokens	['--', 'Dhaka', 'Newsroom', '880-2-506363']
pos_tags	[8, 22, 22, 11]
chunk_tags	[0, 11, 12, 12]
ner_tags	[0, 3, 4, 0]

test
id	3452
tokens	['The', 'lanky', 'former', 'Leeds', 'United', 'defender', 'did', 'not', 'make', 'his', 'England', 'debut', 'until', 'the', 'age', 'of', '30', 'but', 'eventually', 'won', '35', 'caps', 'and', 'was', 'a', 'key', 'member', 'of', 'the', '1966', 'World', 'Cup', 'winning', 'team', 'with', 'his', 'younger', 'brother', ',', 'Bobby', '.']
pos_tags	[12, 16, 16, 22, 22, 21, 38, 30, 37, 29, 22, 21, 15, 12, 21, 15, 11, 10, 30, 40, 11, 24, 10, 38, 12, 16, 21, 15, 12, 11, 22, 22, 16, 21, 15, 29, 17, 21, 6, 22, 7]
chunk_tags	[11, 12, 12, 12, 12, 12, 21, 22, 22, 11, 12, 12, 17, 11, 12, 13, 11, 11, 12, 12, 12, 12, 0, 21, 11, 12, 12, 13, 11, 12, 12, 12, 11, 12, 13, 11, 12, 12, 0, 11, 0]
ner_tags	[0, 0, 0, 3, 4, 

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [8]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [7]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [8]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [9]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,10718,"[VfL, Bochum, 4, 1, 3, 0, 4, 3, 6]","[NNP, NNP, CD, CD, CD, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, I-ORG, O, O, O, O, O, O, O]"
1,7793,"[--, Tosco, Corp, to, $, 0.95, from, $, 1.03, versus, the, consensus, $, 0.94]","[:, NNP, NNP, TO, $, CD, IN, $, CD, IN, DT, NN, $, CD]","[B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, I-NP]","[O, B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O]"
2,1339,"[Other, issues, affecting, ties, include, plans, by, an, Australian, cabinet, minister, to, visit, Taiwan, ,, a, security, pact, between, Canberra, and, Washington, and, a, possible, visit, to, Australia, next, month, by, Tibet, 's, exiled, spiritual, leader, the, Dalai, Lama, .]","[JJ, NNS, VBG, NNS, VBP, NNS, IN, DT, JJ, NN, NN, TO, VB, NNP, ,, DT, JJ, NN, IN, NNP, CC, NNP, CC, DT, JJ, NN, TO, NNP, JJ, NN, IN, NNP, POS, VBD, JJ, NN, DT, NNP, NNP, .]","[B-NP, I-NP, B-VP, B-NP, B-VP, B-NP, B-PP, B-NP, I-NP, I-NP, I-NP, B-VP, I-VP, B-NP, O, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, I-NP, O, B-NP, I-NP, I-NP, B-PP, B-NP, B-NP, I-NP, B-PP, B-NP, I-NP, B-VP, B-NP, I-NP, B-NP, I-NP, I-NP, O]","[O, O, O, O, O, O, O, O, B-MISC, O, O, O, O, B-LOC, O, O, O, O, O, B-LOC, O, B-LOC, O, O, O, O, O, B-LOC, O, O, O, B-LOC, O, O, O, O, O, B-PER, I-PER, O]"
3,4935,"[Sheffield, Wednesday, 3, 3, 0, 0, 6, 2, 9]","[NNP, NNP, CD, CD, CD, CD, CD, CD, CD]","[B-NP, B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, I-ORG, O, O, O, O, O, O, O]"
4,5015,"[LONDON, 1996-08-25]","[NNP, CD]","[B-NP, I-NP]","[B-LOC, O]"
5,12614,"[We, stay, in, Manhattan, and, it, 's, a, long, way, to, come, .]","[PRP, VBP, IN, NNP, CC, PRP, VBZ, DT, JJ, NN, TO, VB, .]","[B-NP, B-VP, B-PP, B-NP, O, B-NP, B-VP, B-NP, I-NP, I-NP, B-VP, I-VP, O]","[O, O, O, B-LOC, O, O, O, O, O, O, O, O, O]"
6,173,"[There, is, no, objection, to, their, travel, ,, "", he, added, .]","[EX, VBZ, DT, NN, TO, PRP$, NN, ,, "", PRP, VBD, .]","[B-NP, B-VP, B-NP, I-NP, B-PP, B-NP, I-NP, O, O, B-NP, B-VP, O]","[O, O, O, O, O, O, O, O, O, O, O, O]"
7,11087,"[He, is, being, asked, for, the, source, of, his, information, and, who, gave, him, this, confidential, document, ,, "", one, colleague, told, Reuters, .]","[PRP, VBZ, VBG, VBN, IN, DT, NN, IN, PRP$, NN, CC, WP, VBD, PRP, DT, JJ, NN, ,, "", CD, NN, VBN, NNP, .]","[B-NP, B-VP, I-VP, I-VP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, O, B-NP, B-VP, B-NP, B-NP, I-NP, I-NP, O, O, B-NP, I-NP, B-VP, B-NP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG, O]"
8,10559,"[SOCCER, -, BARCELONA, WIN, SPANISH, SUPER, CUP, .]","[NN, :, NNP, NNP, NNP, NNP, NNP, .]","[B-NP, O, B-NP, I-NP, I-NP, I-NP, I-NP, O]","[O, O, B-ORG, O, B-MISC, B-MISC, I-MISC, O]"
9,906,"[Strikers, -, Kubilay, Turkyilmaz, (, Grasshoppers, ), ,, Adrian, Knup, (, Galatasaray, ), ,, Christophe, Bonvin, (, Sion, ), ,, Stephane, Chapuisat, (, Borussia, Dortmund, ), .]","[NNS, :, NNP, NNP, (, NNPS, ), ,, NNP, NNP, (, NNP, ), ,, NNP, NNP, (, NNP, ), ,, NNP, NNP, (, NNP, NNP, ), .]","[B-NP, O, B-NP, I-NP, O, B-NP, O, O, B-NP, I-NP, O, B-NP, O, O, B-NP, I-NP, O, B-NP, O, O, B-NP, I-NP, O, B-NP, I-NP, O, O]","[O, O, B-PER, I-PER, O, B-ORG, O, O, B-PER, I-PER, O, B-ORG, O, O, B-PER, I-PER, O, B-ORG, O, O, B-PER, I-PER, O, B-ORG, I-ORG, O, O]"


### Preprocessing the data
Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure that

- we get a tokenizer that corresponds to the model architecture we want to use and that
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [12]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).<br>
You can directly call this tokenizer on one sentence:

In [13]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [102, 10503, 30892, 818, 3070, 1420, 14492, 6661, 155, 730, 3330, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [14]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [102, 10503, 30892, 818, 3070, 1420, 14492, 6661, 155, 730, 428, 18260, 26378, 5928, 5559, 566, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [15]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [16]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'ger', '##man', '##y', "'", 's', 're', '##pres', '##enta', '##tive', 'to', 'the', 'eur', '##ope', '##an', 'un', '##ion', "'", 's', 've', '##ter', '##ina', '##ry', 'comm', '##ittee', 'wer', '##ner', 'zw', '##ing', '##mann', 'said', 'on', 'we', '##dn', '##esday', 'con', '##sum', '##ers', 'sho', '##uld', 'bu', '##y', 'she', '##ep', '##me', '##at', 'from', 'co', '##unt', '##ries', 'other', 'than', 'brit', '##ain', 'un', '##til', 'the', 'sc', '##ient', '##ific', 'ad', '##vic', '##e', 'was', 'cle', '##arer', '.', '[SEP]']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [17]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 68)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [18]:
print(tokenized_input.word_ids())

[None, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 4, 5, 5, 5, 6, 6, 7, 7, 8, 8, 8, 8, 9, 9, 10, 10, 11, 11, 11, 12, 13, 14, 14, 14, 15, 15, 15, 16, 16, 17, 17, 18, 18, 18, 18, 19, 20, 20, 20, 21, 22, 23, 23, 24, 24, 25, 26, 26, 26, 27, 27, 27, 28, 29, 29, 30, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [19]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

68 68


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [20]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [21]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [22]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[102, 11123, 474, 24680, 839, 1306, 9845, 30891, 1512, 7732, 22711, 1044, 3470, 3194, 4268, 2655, 566, 103], [102, 10602, 162, 3398, 1329, 28013, 103], [102, 2118, 10790, 235, 4193, 232, 3167, 232, 1785, 103], [102, 871, 12618, 12809, 114, 19178, 795, 27751, 1158, 1773, 15892, 2021, 9171, 246, 6384, 30888, 5470, 839, 1306, 3668, 23298, 30881, 1512, 6199, 9274, 462, 1512, 9330, 111, 3470, 3194, 4268, 2655, 258, 4238, 10860, 5586, 20226, 21805, 105, 11045, 9299, 12332, 30884, 6138, 30888, 8844, 30899, 9171, 12032, 237, 9051, 248, 13671, 15484, 30888, 1512, 29422, 4054, 566, 103], [102, 839, 1306, 30933, 2119, 117, 474, 21651, 19723, 4883, 1512, 871, 12618, 12809, 114, 258, 220, 2119, 117, 22037, 162, 1422, 2690, 19178, 19881, 309, 432, 544, 323, 892, 27751, 1158, 6686, 5758, 15157, 6199, 9274, 462, 22565, 4177, 22033, 30933, 29422, 4054, 472, 141, 5413, 8844, 2565, 5093, 18101, 25431, 3470, 3606, 258, 4238, 871, 10860, 5586, 15181, 3668, 23298, 30881, 556, 19637, 27087, 56

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [23]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-6c2f7f0009b95b0d.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-4ffeafbfde9ae079.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-a8e730d37603cce3.arrow


In [39]:
print(len(tokenized_datasets["train"]["id"]))
#tokenized_datasets["train"].features
tokenized_datasets["train"]

14042


Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 14042
})

In [41]:
my_tokenized_train_dataset = tokenized_datasets["train"].features
my_tokenized_train_dataset.num_rows = len(tokenized_datasets["train"]["id"])

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model
Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [42]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    ignore_mismatched_sizes=True # https://github.com/huggingface/transformers/issues/13127 (answer by "NielsRogge")
)

loading configuration file https://huggingface.co/fhswf/bert_de_ner/resolve/main/config.json from cache at /home/matthias/.cache/huggingface/transformers/96487e57471d7f54f5f5f8345e4edcdafe6b12838d95aac3272abee90cd0ffa4.69603aa173b38e6ea173f48e89e2c02113d17adedb7addfe120a080aaaeb5b1f
Model config BertConfig {
  "_name_or_path": "fhswf/bert_de_ner",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [43]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [44]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [45]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [46]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [47]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [48]:
trainer = Trainer(
    model,
    args,
    train_dataset=my_tokenized_train_dataset,#tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/matthias/Desktop/Project/cs224u/base_data/Matthias/bert_de_ner-finetuned-ner is already a clone of https://huggingface.co/mdroth/bert_de_ner-finetuned-ner. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

### Zero-shot baseline

In [49]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, chunk_tags, tokens, pos_tags, ner_tags.
***** Running Evaluation *****
  Num examples = 3251
  Batch size = 8


{'eval_loss': 2.435607671737671,
 'eval_precision': 0.026918466488612226,
 'eval_recall': 0.08346918813172481,
 'eval_f1': 0.040708583786534365,
 'eval_accuracy': 0.04756687077022236,
 'eval_runtime': 10.8411,
 'eval_samples_per_second': 299.877,
 'eval_steps_per_second': 37.542}

### Training
We can now finetune our model by just calling the `train` method:

In [50]:
trainer.train()

/home/matthias/anaconda3/envs/hf20220408/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6


KeyError: 1

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [35]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, chunk_tags, pos_tags, id.
***** Running Evaluation *****
  Num examples = 3251
  Batch size = 8


{'eval_loss': 0.12525352835655212,
 'eval_precision': 0.8956567448584006,
 'eval_recall': 0.8888816432996414,
 'eval_f1': 0.8922563330496824,
 'eval_accuracy': 0.9691481362122677,
 'eval_runtime': 10.8324,
 'eval_samples_per_second': 300.119,
 'eval_steps_per_second': 37.573,
 'epoch': 3.0}

## Inference
To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [36]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags, chunk_tags, pos_tags, id.
***** Running Prediction *****
  Num examples = 3251
  Batch size = 8


{'LOC': {'precision': 0.9122771880632886,
  'recall': 0.9303513071895425,
  'f1': 0.9212256042066943,
  'number': 4896},
 'MISC': {'precision': 0.8460537727666956,
  'recall': 0.8460537727666956,
  'f1': 0.8460537727666955,
  'number': 2306},
 'ORG': {'precision': 0.8567599761762954,
  'recall': 0.818259385665529,
  'f1': 0.8370672097759675,
  'number': 3516},
 'PER': {'precision': 0.9311705392371766,
  'recall': 0.9200779727095516,
  'f1': 0.9255910229872535,
  'number': 4617},
 'overall_precision': 0.8956567448584006,
 'overall_recall': 0.8888816432996414,
 'overall_f1': 0.8922563330496824,
 'overall_accuracy': 0.9691481362122677}